In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
import pickle
import os


def log(msg):
    print(msg, '\n')


def createFolders():
    for breed in list(classDic.keys()):
        # Breeds
        trainPath = os.path.join('./breeds/train/' + breed)
        if not os.path.exists(trainPath):
            os.mkdir('./breeds/train/' + breed)

        testPath = os.path.join('./breeds/test/' + breed)
        if not os.path.exists(testPath):
            os.mkdir('./breeds/test/' + breed)

        # Dogs
        trainPath = os.path.join('./species/train/dog')
        if not os.path.exists(trainPath):
            os.mkdir('./species/train/dog')

        testPath = os.path.join('./species/test/dog')
        if not os.path.exists(testPath):
            os.mkdir('./species/test/dog')

        # Cats
        trainPath = os.path.join('./species/train/cat')
        if not os.path.exists(trainPath):
            os.mkdir('./species/train/cat')

        testPath = os.path.join('./species/test/cat')
        if not os.path.exists(testPath):
            os.mkdir('./species/test/cat')


def splitBreeds():
    for imgName in list(filesDic.keys()):
        img = keras.preprocessing.image.load_img('images/' + imgName)
        foldTrain = filesDic.get(imgName).get('foldTrain')

        keras.preprocessing.image.save_img(
            './breeds/' + ('train/' if foldTrain else 'test/') + filesDic.get(imgName).get('breed') + '/' + imgName, img)


def splitSpecies():
    for imgName in list(filesDic.keys()):
        img = keras.preprocessing.image.load_img('images/' + imgName)
        foldTrain = filesDic.get(imgName).get('foldTrain')

        keras.preprocessing.image.save_img(
            './species/' + ('train/' if foldTrain else 'test/') + filesDic.get(imgName).get('species') + '/' + imgName, img)


D = dict(pickle.load(open('Oxford-IIIT-Pet_Dics.p', 'rb')))
# log(D)
imageNum = 7390

DKeys = list(D.keys())
# log(DKeys)

classDic = dict(D.get('classDic'))
# log(classDic)

filesDic = dict(D.get('filesDic'))
# log(list(filesDic.keys()))
filesDicValues = list(filesDic.values())
# log(filesDicValues[-1:])

# createFolders()
# splitBreeds()
# splitSpecies()


# CNN network from scratch

In [ ]:
speciesCnn = models.Sequential()

speciesCnn.add(layers.Conv2D(6, (5, 5), activation='relu',
                             input_shape=(224, 224, 3), padding="same"))
speciesCnn.add(layers.AveragePooling2D((2, 2)))
speciesCnn.add(layers.Conv2D(16, (5, 5), activation='relu'))
speciesCnn.add(layers.AveragePooling2D((2, 2)))
speciesCnn.add(layers.Conv2D(120, (1, 1), activation='relu'))
speciesCnn.add(layers.Flatten())
speciesCnn.add(layers.Dense(64, activation='relu'))
speciesCnn.add(layers.Dense(2, activation='softmax'))  # 2 species

speciesCnn.summary()

speciesCnn.compile(optimizer="nadam",
                   loss="categorical_crossentropy",
                   metrics=["accuracy"])


In [ ]:
breedsCnn = models.Sequential()

breedsCnn.add(layers.Conv2D(6, (5, 5), activation='relu',
                            input_shape=(224, 224, 3), padding="same"))
breedsCnn.add(layers.AveragePooling2D((2, 2)))
breedsCnn.add(layers.Conv2D(16, (5, 5), activation='relu'))
breedsCnn.add(layers.AveragePooling2D((2, 2)))
breedsCnn.add(layers.Conv2D(120, (1, 1), activation='relu'))
breedsCnn.add(layers.Flatten())
breedsCnn.add(layers.Dense(64, activation='relu'))
breedsCnn.add(layers.Dense(37, activation='softmax'))  # 37 breeds

breedsCnn.summary()

breedsCnn.compile(optimizer="nadam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])


We're using input size of 224x224 because it seems a good shape for the overall images and it's to be compatible with the mobileNet network

# Pre trained network

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

mn2 = MobileNetV2(weights='imagenet',
                  input_shape=(224, 224, 3))

mn2.trainable = False

# mn2.summary()

breedsMn2Cnn = models.Sequential()
breedsMn2Cnn.add(mn2)
breedsMn2Cnn.add(layers.Flatten())
breedsMn2Cnn.add(layers.Dense(37, activation='softmax'))  # 37 breeds

breedsMn2Cnn.summary()

breedsMn2Cnn.compile(optimizer="nadam",
                     loss="categorical_crossentropy",
                     metrics=["accuracy"])


We have chosen the mobileNetV2 because it's a network that has 3.5M parameters and makes it quick for us to train, compared to other networks.

# Data generator with and without data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

breedsDir = './breeds/'

breedsGen = ImageDataGenerator(preprocessing_function=preprocess_input)

breedsGenAug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode="nearest")

breedsTrainGen = breedsGen.flow_from_directory(directory=breedsDir + "train/",
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               batch_size=32)

breedsTrainGenAug = breedsGenAug.flow_from_directory(directory=breedsDir + "train/",
                                                     target_size=(224, 224),
                                                     class_mode="categorical",
                                                     batch_size=32)

breedsTestGen = breedsGen.flow_from_directory(directory=breedsDir + "test/",
                                              target_size=(224, 224),
                                              class_mode="categorical",
                                              batch_size=32)

speciesDir = './species/'

speciesGen = ImageDataGenerator(preprocessing_function=preprocess_input)

speciesGenAug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

speciesTrainGen = speciesGen.flow_from_directory(directory=speciesDir + "train/",
                                                 target_size=(224, 224),
                                                 class_mode="categorical",
                                                 batch_size=32)

speciesTrainGenAug = speciesGenAug.flow_from_directory(directory=speciesDir + "train/",
                                                       target_size=(224, 224),
                                                       class_mode="categorical",
                                                       batch_size=32)

speciesTestGen = speciesGen.flow_from_directory(directory=speciesDir + "test/",
                                                target_size=(224, 224),
                                                class_mode="categorical",
                                                batch_size=32)


Here we don't augment the data for the test set

If we were to use validation data, we wouldn't use data augmentation either



# Binary (Species)

## Without data augmentation

In [ ]:
log = speciesCnn.fit(speciesTrainGen,
                     epochs=1,
                     validation_data=speciesTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = speciesCnn.predict(speciesTestGen)
print(predicted)


## With data augmentation

In [ ]:
log = speciesCnn.fit(speciesTrainGenAug,
                     epochs=25,
                     validation_data=speciesTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = speciesCnn.predict(speciesTestGen)
print(predicted)


# Multi-class (Breeds)

## Without data augmentation

### Scratch network

In [ ]:
log = breedsCnn.fit(breedsTrainGen,
                    epochs=25,
                    validation_data=breedsTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = breedsCnn.predict(breedsTestGen)
print(predicted)


### MobileNetV2 network

In [ ]:
log = breedsMn2Cnn.fit(breedsTrainGen,
                       epochs=25,
                       validation_data=breedsTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = breedsMn2Cnn.predict(breedsTestGen)
print(predicted)


# Multi-class (Breeds)

## With data augmentation

### Scratch network

In [ ]:
log = breedsCnn.fit(breedsTrainGenAug,
                    epochs=25,
                    validation_data=breedsTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = breedsCnn.predict(breedsTestGen)
print(predicted)


### MobileNetV2 network

In [ ]:
log = breedsMn2Cnn.fit(breedsTrainGenAug,
                       epochs=25,
                       validation_data=breedsTestGen)

h = log.history
plt.plot(h["loss"], label='Loss')
plt.plot(h["accuracy"], label='Accuracy')
plt.plot(h["val_loss"], label='Val Loss')
plt.plot(h["val_accuracy"], label='Val Acurracy')
plt.legend()
plt.show()

predicted = breedsMn2Cnn.predict(breedsTestGen)
print(predicted)
